# Расчетно-графическое домашнее задание

### Выполнил: Александров А. Н. ИУ8-104
### Вариант: 1

## Задание
Задана платежная матрица прямоугольной игры с нулевой суммой.


| 4  | -3 | 5  | 6  | 4 |
|:--:|:--:|:--:|:--:|:-:|
| 6  | 5  | -3 | 4  | 7 |
| 6  | 5  | -3 | -3 | 5 |
| -3 | -3 | 4  | 4  | 4 |
| 7  | 6  | 4  | 5  | 6 |


1. Нормализовать матрицу (привести к матрице с неотрицательными элементами) и свести исходную игру к матричной игре 2×2 следующими способами:
   - поглощением доминируемых стратегий;
   - удалением NBR-стратегий (Never Best Response).
2. Найти смешанные стратегии игроков следующими методами:
   - графоаналитическим;
   - аналитическим (матричным);
   - графически (задача ЛП);
   - симплекс-методом (задача ЛП).
3. Рассчитать цену игры для исходной матрицы.


In [1]:
"""Dependencies."""
import json
from pathlib import Path

### 1. Нормализация матрицы

#### 1.1. Поглощение доминируемых стратегий

**Доминируемая (поглощающая) строка** содержит элементы >= элементам другой строки (поглощаемой).

**Доминируемый (поглощающая) столбец** содержит элементы <= элементам другого столбца (поглощаемого).